# 策略优化方法

## 📋 概述

策略优化是量化交易中提高策略表现的关键步骤，通过调整策略参数、改进策略逻辑来提升策略的收益风险比。本章节将详细介绍策略优化的方法、优化目标、优化算法以及如何避免过拟合。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际代码示例理解策略优化的方法。


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib scipy akshare

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, differential_evolution
from itertools import product

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 📖 核心内容

### 1. 策略优化的基本概念

#### 1.1 什么是策略优化

**策略优化（Strategy Optimization）**：
- 调整策略参数以提升表现
- 改进策略逻辑
- 平衡收益和风险

**优化的目标**：
- 提高收益率
- 降低风险
- 提高夏普比率
- 降低最大回撤

#### 1.2 优化方法

**参数优化**：
- 网格搜索
- 随机搜索
- 优化算法（遗传算法、粒子群等）

**逻辑优化**：
- 改进信号生成逻辑
- 优化仓位管理
- 改进风险控制

让我们先创建一个简单的策略和回测函数：


In [ ]:
# 简单的回测函数（用于优化）
def simple_backtest(data, short_window, long_window, initial_capital=1000000):
    """
    简单的回测函数
    
    参数:
    data: 价格数据
    short_window: 短期均线周期
    long_window: 长期均线周期
    initial_capital: 初始资金
    """
    # 计算均线
    data = data.copy()
    data['MA_Short'] = data['Close'].rolling(window=short_window).mean()
    data['MA_Long'] = data['Close'].rolling(window=long_window).mean()
    data['Signal'] = 0
    data.loc[data['MA_Short'] > data['MA_Long'], 'Signal'] = 1
    data.loc[data['MA_Short'] < data['MA_Long'], 'Signal'] = -1
    
    # 模拟交易
    capital = initial_capital
    position = 0
    equity_curve = []
    
    for i in range(len(data)):
        price = data['Close'].iloc[i]
        signal = data['Signal'].iloc[i]
        
        if signal > 0 and position == 0:  # 买入
            position = capital / price
            capital = 0
        elif signal < 0 and position > 0:  # 卖出
            capital = position * price
            position = 0
        
        portfolio_value = capital + position * price
        equity_curve.append(portfolio_value)
    
    # 计算指标
    final_value = equity_curve[-1]
    total_return = (final_value - initial_capital) / initial_capital
    
    # 计算年化收益率
    days = len(data)
    years = days / 252.0
    annual_return = (1 + total_return) ** (1 / years) - 1 if years > 0 else 0
    
    # 计算波动率
    returns = pd.Series(equity_curve).pct_change().dropna()
    volatility = returns.std() * np.sqrt(252)
    
    # 计算夏普比率
    risk_free_rate = 0.03
    sharpe_ratio = (annual_return - risk_free_rate) / volatility if volatility > 0 else 0
    
    # 计算最大回撤
    equity_array = np.array(equity_curve)
    cumulative_max = np.maximum.accumulate(equity_array)
    drawdown = (equity_array - cumulative_max) / cumulative_max
    max_drawdown = drawdown.min()
    
    return {
        'total_return': total_return,
        'annual_return': annual_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'volatility': volatility
    }

# 获取数据
try:
    import akshare as ak
    stock_data = ak.stock_zh_a_hist(symbol="000001", period="daily", 
                                     start_date="20230101", end_date="20241231", 
                                     adjust="qfq")
    stock_data.columns = ['日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率']
    stock_data['日期'] = pd.to_datetime(stock_data['日期'])
    stock_data = stock_data.set_index('日期')
    stock_data = stock_data[['开盘', '最高', '最低', '收盘', '成交量']]
    stock_data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    stock_data = stock_data.sort_index()
    print("数据获取成功！")
except Exception as e:
    print(f"数据获取失败：{e}，使用模拟数据")
    dates = pd.date_range(start='2023-01-01', end='2024-12-31', freq='D')
    dates = dates[dates.weekday < 5]
    np.random.seed(42)
    prices = 10 + np.cumsum(np.random.randn(len(dates)) * 0.1)
    stock_data = pd.DataFrame({
        'Open': prices + np.random.randn(len(dates)) * 0.05,
        'High': prices + np.abs(np.random.randn(len(dates)) * 0.1),
        'Low': prices - np.abs(np.random.randn(len(dates)) * 0.1),
        'Close': prices + np.random.randn(len(dates)) * 0.05,
        'Volume': np.random.randint(1000000, 10000000, len(dates))
    }, index=dates)
    stock_data['High'] = stock_data[['Open', 'Close', 'High']].max(axis=1)
    stock_data['Low'] = stock_data[['Open', 'Close', 'Low']].min(axis=1)

print(f"数据准备完成，共{len(stock_data)}条记录")


In [ ]:
# 网格搜索优化
def grid_search_optimization(data, short_range, long_range, objective='sharpe'):
    """
    网格搜索优化
    
    参数:
    data: 价格数据
    short_range: 短期均线周期范围 (start, end, step)
    long_range: 长期均线周期范围 (start, end, step)
    objective: 优化目标 ('sharpe', 'return', 'drawdown')
    """
    short_start, short_end, short_step = short_range
    long_start, long_end, long_step = long_range
    
    short_windows = range(short_start, short_end + 1, short_step)
    long_windows = range(long_start, long_end + 1, long_step)
    
    results = []
    
    for short, long in product(short_windows, long_windows):
        if short >= long:  # 短期必须小于长期
            continue
        
        try:
            metrics = simple_backtest(data, short, long)
            
            # 根据目标函数选择指标
            if objective == 'sharpe':
                score = metrics['sharpe_ratio']
            elif objective == 'return':
                score = metrics['annual_return']
            elif objective == 'drawdown':
                score = -metrics['max_drawdown']  # 负号因为要最小化回撤
            else:
                score = metrics['sharpe_ratio']
            
            results.append({
                'short_window': short,
                'long_window': long,
                'sharpe_ratio': metrics['sharpe_ratio'],
                'annual_return': metrics['annual_return'],
                'max_drawdown': metrics['max_drawdown'],
                'volatility': metrics['volatility'],
                'score': score
            })
        except Exception as e:
            continue
    
    results_df = pd.DataFrame(results)
    if len(results_df) > 0:
        best_params = results_df.loc[results_df['score'].idxmax()]
        return results_df, best_params
    else:
        return pd.DataFrame(), None

# 运行网格搜索
print("开始网格搜索优化...")
results_df, best_params = grid_search_optimization(
    stock_data, 
    short_range=(3, 10, 1),
    long_range=(15, 30, 2),
    objective='sharpe'
)

if best_params is not None:
    print(f"\n最优参数：")
    print(f"短期均线：{best_params['short_window']:.0f}")
    print(f"长期均线：{best_params['long_window']:.0f}")
    print(f"夏普比率：{best_params['sharpe_ratio']:.4f}")
    print(f"年化收益率：{best_params['annual_return']*100:.2f}%")
    print(f"最大回撤：{best_params['max_drawdown']*100:.2f}%")
else:
    print("未找到有效参数组合")


### 3. 优化结果可视化

让我们可视化优化结果：


In [ ]:
# 可视化优化结果
if len(results_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. 参数热力图（夏普比率）
    pivot_sharpe = results_df.pivot(index='long_window', columns='short_window', values='sharpe_ratio')
    im1 = axes[0, 0].imshow(pivot_sharpe, aspect='auto', cmap='RdYlGn', origin='lower')
    axes[0, 0].set_title('夏普比率热力图', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('短期均线周期')
    axes[0, 0].set_ylabel('长期均线周期')
    axes[0, 0].set_xticks(range(len(pivot_sharpe.columns)))
    axes[0, 0].set_xticklabels(pivot_sharpe.columns)
    axes[0, 0].set_yticks(range(len(pivot_sharpe.index)))
    axes[0, 0].set_yticklabels(pivot_sharpe.index)
    plt.colorbar(im1, ax=axes[0, 0], label='夏普比率')
    
    # 2. 参数热力图（年化收益率）
    pivot_return = results_df.pivot(index='long_window', columns='short_window', values='annual_return')
    im2 = axes[0, 1].imshow(pivot_return, aspect='auto', cmap='RdYlGn', origin='lower')
    axes[0, 1].set_title('年化收益率热力图', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('短期均线周期')
    axes[0, 1].set_ylabel('长期均线周期')
    axes[0, 1].set_xticks(range(len(pivot_return.columns)))
    axes[0, 1].set_xticklabels(pivot_return.columns)
    axes[0, 1].set_yticks(range(len(pivot_return.index)))
    axes[0, 1].set_yticklabels(pivot_return.index)
    plt.colorbar(im2, ax=axes[0, 1], label='年化收益率')
    
    # 3. 夏普比率分布
    axes[1, 0].hist(results_df['sharpe_ratio'], bins=20, color='skyblue', alpha=0.7, edgecolor='black')
    axes[1, 0].axvline(x=best_params['sharpe_ratio'], color='red', linestyle='--', 
                      linewidth=2, label=f'最优值: {best_params["sharpe_ratio"]:.4f}')
    axes[1, 0].set_title('夏普比率分布', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('夏普比率')
    axes[1, 0].set_ylabel('频数')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 收益-风险散点图
    scatter = axes[1, 1].scatter(results_df['volatility']*100, results_df['annual_return']*100,
                                c=results_df['sharpe_ratio'], cmap='RdYlGn', 
                                s=100, alpha=0.6, edgecolors='black')
    axes[1, 1].scatter(best_params['volatility']*100, best_params['annual_return']*100,
                      color='red', s=300, marker='*', edgecolors='black', 
                      linewidth=2, label='最优参数', zorder=5)
    axes[1, 1].set_title('收益-风险散点图', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('年化波动率 (%)')
    axes[1, 1].set_ylabel('年化收益率 (%)')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=axes[1, 1], label='夏普比率')
    
    plt.tight_layout()
    plt.show()


### 4. 优化算法

除了网格搜索，我们还可以使用优化算法来寻找最优参数：


In [ ]:
# 使用优化算法
def objective_function(params, data):
    """目标函数（负夏普比率，因为要最小化）"""
    short, long = int(params[0]), int(params[1])
    if short >= long or short < 1 or long < 1:
        return 1e10  # 返回一个很大的值
    
    try:
        metrics = simple_backtest(data, short, long)
        return -metrics['sharpe_ratio']  # 负号因为要最小化
    except:
        return 1e10

# 使用差分进化算法优化
print("\n使用差分进化算法优化...")
bounds = [(3, 10), (15, 30)]  # 参数范围

result = differential_evolution(
    objective_function,
    bounds,
    args=(stock_data,),
    seed=42,
    maxiter=50,
    popsize=10
)

if result.success:
    optimal_short = int(result.x[0])
    optimal_long = int(result.x[1])
    optimal_metrics = simple_backtest(stock_data, optimal_short, optimal_long)
    
    print(f"\n优化算法结果：")
    print(f"短期均线：{optimal_short}")
    print(f"长期均线：{optimal_long}")
    print(f"夏普比率：{optimal_metrics['sharpe_ratio']:.4f}")
    print(f"年化收益率：{optimal_metrics['annual_return']*100:.2f}%")
    print(f"最大回撤：{optimal_metrics['max_drawdown']*100:.2f}%")
else:
    print("优化失败")


### 5. 避免过拟合

#### 5.1 过拟合的原因

**过拟合（Overfitting）**：
- 策略在训练数据上表现好
- 但在新数据上表现差
- 参数过度优化导致

#### 5.2 防止过拟合的方法

**样本外测试**：
- 将数据分为训练集和测试集
- 在训练集上优化
- 在测试集上验证

**交叉验证**：
- 多次划分训练集和测试集
- 评估策略的稳健性

**参数稳定性**：
- 检查参数附近的性能
- 避免选择极端参数

让我们实现样本外测试：


In [ ]:
# 样本外测试
def out_of_sample_test(data, train_ratio=0.7):
    """
    样本外测试
    
    参数:
    data: 价格数据
    train_ratio: 训练集比例
    """
    # 划分训练集和测试集
    split_idx = int(len(data) * train_ratio)
    train_data = data.iloc[:split_idx]
    test_data = data.iloc[split_idx:]
    
    print(f"训练集：{len(train_data)}条，测试集：{len(test_data)}条")
    
    # 在训练集上优化
    print("\n在训练集上优化...")
    train_results, best_train = grid_search_optimization(
        train_data,
        short_range=(3, 10, 1),
        long_range=(15, 30, 2),
        objective='sharpe'
    )
    
    if best_train is not None:
        # 在测试集上验证
        print("\n在测试集上验证...")
        test_metrics = simple_backtest(
            test_data,
            int(best_train['short_window']),
            int(best_train['long_window'])
        )
        
        print(f"\n训练集结果：")
        print(f"短期均线：{best_train['short_window']:.0f}")
        print(f"长期均线：{best_train['long_window']:.0f}")
        print(f"夏普比率：{best_train['sharpe_ratio']:.4f}")
        print(f"年化收益率：{best_train['annual_return']*100:.2f}%")
        
        print(f"\n测试集结果：")
        print(f"夏普比率：{test_metrics['sharpe_ratio']:.4f}")
        print(f"年化收益率：{test_metrics['annual_return']*100:.2f}%")
        print(f"最大回撤：{test_metrics['max_drawdown']*100:.2f}%")
        
        # 比较训练集和测试集
        print(f"\n性能差异：")
        print(f"夏普比率差异：{test_metrics['sharpe_ratio'] - best_train['sharpe_ratio']:.4f}")
        print(f"收益率差异：{(test_metrics['annual_return'] - best_train['annual_return'])*100:.2f}%")
        
        return best_train, test_metrics
    
    return None, None

# 运行样本外测试
best_train, test_metrics = out_of_sample_test(stock_data)


## 💡 关键要点总结

1. **优化方法**：网格搜索、随机搜索、优化算法
2. **优化目标**：夏普比率、收益率、最大回撤等
3. **过拟合问题**：策略在训练集表现好，测试集表现差
4. **防止过拟合**：样本外测试、交叉验证、参数稳定性检查

## 🛠️ 工具与软件

### 优化工具

- **scipy.optimize**：Python优化库
- **sklearn**：机器学习库，包含交叉验证
- **自定义优化**：根据需求实现

### 注意事项

- ✅ **样本外测试**：必须在样本外数据上验证
- ✅ **避免过度优化**：不要过度拟合历史数据
- ✅ **参数稳定性**：检查参数附近的性能
- ⚠️ **过拟合风险**：优化结果可能过度拟合
- ⚠️ **计算成本**：优化可能需要较长时间

## 🔗 相关知识点

- [策略设计原理](./策略设计原理.ipynb)
- [策略回测方法](./策略回测方法.ipynb)
- [量化交易系统架构](../01_量化交易基础/量化交易系统架构.ipynb)

## 📚 拓展阅读

- 《量化交易：如何建立自己的算法交易》- 策略优化
- 《Python量化交易实战》- 实战案例
- 优化算法相关书籍 - 深入理解优化方法
